In [2]:
import sys
sys.path.append('..')
from seismic_postprocessing import *

data_path = 'data/'
min_coverage = 3000

# ref2plate = get_ref2plate('data/S029472-R1_SynbioTechnologies.xlsx')
seismic_df = read_seismic_output(output_path=os.path.join(data_path, 'out'), ref2plate=None, min_cov=min_coverage)

Number of references in output:  268


In [3]:
filtered_df = filter_bad_references(seismic_df, min_reads=min_coverage, signal_tresholds=(0.3, 0.7), min_covered=0.5, primers=None)
plot_signal_histogram(filtered_df)

-> Before filtering: Number of sequences: 209 and number of unique sequences: 101
-> After filtering low reads: Number of sequences: 128 and number of unique sequences: 60
-> After filtering high mutations: Number of sequences: 109 and number of unique sequences: 52
(1 references were updated and 8 references were removed)
-> After filtering not well covered: Number of sequences: 86 and number of unique sequences: 40


In [15]:
sub_A = np.concatenate(filtered_df[filtered_df['replicate'] == 'A'].sub_rate.tolist())
sub_A = sub_A[sub_A >= 0]
print(sub_A.mean(), sub_A.std())

sub_B = np.concatenate(filtered_df[filtered_df['replicate'] == 'B'].sub_rate.tolist())
sub_B = sub_B[sub_B >= 0]
print(sub_B.mean(), sub_B.std())

sub_Un = np.concatenate(filtered_df[filtered_df['replicate'] == 'Untreated'].sub_rate.tolist())
sub_Un = sub_Un[sub_Un >= 0]
print(sub_Un.mean(), sub_Un.std())

0.011091102901196487 0.06458659555887812
0.015616444036057102 0.037563693558513266
0.005906504302361346 0.019454961263290087


In [4]:
filtered_df[filtered_df['replicate']!='Untreated'].reset_index().to_feather(os.path.join(data_path, 'processed', 'mRNA_filteredAllbases.feather'))

In [ ]:
filtered_df.reset_index().to_feather(os.path.join(data_path, 'processed', 'mRNA_filteredAllbases.feather'))

In [4]:
normalized_df = normalize_dms(filtered_df, percentile=95)
print_summary(normalized_df)
plot_histogram(normalized_df)

0 :  86
Total:  86


In [17]:
combined_df = combine_replicates(normalized_df, min_correlation=0.0, )
plot_correlations_per_plate(combined_df, mode='correlation')
print_summary(combined_df)

0 :  29
Total:  29


In [64]:
data_results = pd.read_feather('/Users/alberic/Downloads/test_results_PT+FT.feather')

data_results = data_results[data_results.sequence.str.len()<50]

base_pairing_matrix = np.vstack(data_results.structure.values[10])

In [62]:
23*23

529

In [47]:
data_results = pd.read_feather('/Users/alberic/Downloads/test_results_PT+FT.feather')

data_results[data_results.sequence.str.len()<50]

,reference,sequence,structure
0,2N7X-2D,GGUAGUUUUGGCAUGACUCUACC,"[[-8.36166000366211, -9.895757675170898, -11.2..."
2,2CD1-2D,GGAAGUCCGGUCUUCGGACCGGCUUCC,"[[-15.138818740844727, -19.11273956298828, -21..."
3,1WTT-2D,GGACCGGAAGGUCC,"[[-14.469758987426758, -19.25482940673828, -22..."
4,2NCI-2D,GGACGUUAAAAGGCUUCGGCCUACGUCC,"[[-14.88586139678955, -18.460533142089844, -21..."
5,2Y95-2D,GGCGCAUCGGCGCC,"[[-15.212677001953125, -19.035137176513672, -2..."
...,...,...,...
351,2LU0-2D,GGAAUAUGCUCAACGAAAGUGAAUCAGCUUCGGCUGAGAGCUAAGUUCC,"[[-6.815932273864746, -8.569583892822266, -8.8..."
352,2M12-2D,GGGUGUAUUGGAAAUGAGCACCC,"[[-8.359880447387695, -10.309768676757812, -11..."
353,2KHY-2D,GGGAUCACAAGUAGGACUUCGGUCCGAAUACAGAUCUC,"[[-8.128787994384766, -9.106486320495605, -11...."
354,1R7W-2D,GGAGGACAUCCCUCACGGGUGACCGUGGUCCUCC,"[[-9.03823184967041, -10.824990272521973, -12...."


In [22]:
combined_df.loc[combined_df.r2.argmax()]

sample         [Un_1_S10_L001, 1-5_1_B_S12_L001, 1-5_1_A_S11_...
reference                                     ENSG00000106263.19
plate                                                          0
sequence       GGGCCGTGTGTGCTGTGGAGCCGAGGCCGTCCTGCAGGAAGCCGCG...
sub_rate       [-1000.0, -1000.0, -1000.0, 0.0, 0.00010952057...
coverage       [-1000.0, -1000.0, -1000.0, 478072.02499999997...
correlation                                             0.472805
r2                                                     -0.022894
Name: 20, dtype: object

In [7]:
# Output data
output_path = os.path.join(data_path, 'processed')
if os.path.exists(output_path):
    os.system('rm -r {}'.format(output_path))
    
plot_dms_signal(combined_df, output_path)
dump_df_to_dreem_format(combined_df, name = 'human_mRNA', path=output_path, 
                        metadata={  '#Species': 'human',
                                    '#Family': 'mRNA',
                                    '#Experimental_method': 'DMS-MaPseq',
                                    '#Temperature': '37',
                                    '#Concentration': '1.5',
                                    "#Condition": 'in vitro'})

# RouskinHF

In [8]:
import envbash
envbash.load.load_envbash('../.env')
from rouskinhf import convert, upload_dataset, get_dataset

In [10]:
data = convert(
    'seismic',
    'data/processed/human_mRNA.json',
    name='human_mRNA',
    path_out='data',
    filter=True,
    predict_structure=True
)

Parsing dreem output file: 100%|██████████| 1503/1503 [5:14:34<00:00, 12.56s/it]    


Over a total of 1503 datapoints, there are:
### OUTPUT
- ALL: 1456 valid datapoints
- INCLUDED: 0 duplicate sequences with different structure / dms / shape
### MODIFIED
- 0 multiple sequences with the same reference (renamed reference)
### FILTERED OUT
- 0 invalid datapoints (ex: sequence with non-regular characters)
- 0 datapoints with bad structures
- 0 duplicate sequences with the same structure / dms / shape
- 47 datapoints removed because of low AUROC (<0.8)


## To HuggingFace

In [11]:
upload_dataset(
    'data/human_mRNA/data.json',
    commit_message='changed UTR to human_mRNA',
    exist_ok=True,
)

data.json:   0%|          | 0.00/11.9M [00:00<?, ?B/s]